In [5]:
from tqdm.auto import tqdm
from osgeo import gdal, ogr
import geopandas as gpd
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
from pyproj import Proj, Transformer
import rasterio

# Data crop

In [6]:
def crop_image_using_coordinates(InputImage, OutputImage, RefImage, latitude, longitude, CropSize):\
    
    #참조 이미지의 boundary를 가져옴 
    Image = gdal.Open(RefImage, gdal.GA_ReadOnly)
    width = Image.RasterXSize
    height = Image.RasterYSize
    Image = None

    """extract vertex coordinates"""
    rds = rasterio.open(RefImage)
    rds.bounds
    left = rds.bounds[0]
    right = rds.bounds[2]
    top = rds.bounds[3]
    bottom = rds.bounds[1]

   
    resolution_x = (right - left) / width
    resolution_y = (top - bottom) / height

    InputCrs = 'EPSG:4326'
    OutputCrs ='EPSG:4326'
    transformer = Transformer.from_crs(InputCrs, OutputCrs)
    longitude, latitude = transformer.transform(longitude, latitude)
    
    left_box = latitude - (resolution_x * CropSize)
    top_box = longitude + (resolution_y * CropSize)
    right_box = latitude + (resolution_x * CropSize)
    bottom_box = longitude - (resolution_y * CropSize)
    window = (left_box, top_box, right_box, bottom_box)

    gdal.Translate(OutputImage, InputImage, projWin = window)

# Train land data crop

In [ ]:
def crop_train(data_n,o_data,crop_size):
    filepath=f'../data/geo_data/crop/train/{data_n}'
    os.makedirs(filepath, exist_ok=True)
    
    InputImage = f'../data/geo_data/raw/{data_n}_test.tif'   
    RefImage = '../data/gw_boundary/boundary_blank_resized.tif'
    
    for i in tqdm(range(len(o_data))):
        OutputImage = filepath+'/Crop_'+data_n+'_'+str(i)+'.tif'
        
        lon=o_data['lon'][i]
        lat=o_data['lat'][i]
        
        crop_image_using_coordinates(InputImage, OutputImage, RefImage, lon, lat, crop_size)
    print(f"{data_n} Train data crop complete")

In [9]:
train_data=pd.read_csv("../data/train_data.csv")

In [23]:
crop_train('Height',train_data,1)
#crop_train('Height',train_data,1)
#crop_train('Slope',train_data,1)
#crop_train('Landuse',train_data,1)
#crop_train('population_density',train_data,1)
#crop_train('NDVI',train_data,1)

  0%|          | 0/2193 [00:00<?, ?it/s]

Height Train data crop complete


# Test land data crop

In [3]:
N = 38.61370931
E = 129.359995
S = 37.03353708
W = 127.0950376

width = E-W
height= N-S
width_len=width/399
height_len=height/277

In [ ]:
width_num,height_num=[], []
for i in range(400):
    width_num.append(W+width_len*i)
width_num=[round(i,7) for i in width_num]
for i in range(278):
    height_num.append(N-height_len*i)
height_num=[round(i,7) for i in height_num]

In [ ]:
RefImage = '../data/gw_boundary/boundary_blank_resized.tif'

data_n=["Slope",'Height','Landuse','population_density','NDVI']

for i in range(len(data_n)):
    filepath=f'../data/geo_data/crop/test/{data_n}'
    os.makedirs(filepath, exist_ok=True)
    num=0
    for lat in height_num:
        for lon in width_num:
            OutputImage=filepath+data_n[i]+'/'+data_n[i]+'_crop'+str(num)
            num=num+1
            crop_image_using_coordinates(InputImage, OutputImage, RefImage, lon, lat, 1)

# Crop data to npy

In [ ]:
Height_train= natsort.natsorted(glob(f"C:/Users/EBDL/Desktop/Land/re_train/train/height/*.tif"))
NDVI_train= natsort.natsorted(glob(f"C:/Users/EBDL/Desktop/Land/re_train/train/ndvi/*.tif"))
Slope_train= natsort.natsorted(glob(f"C:/Users/EBDL/Desktop/Land/re_train/train/slope/*.tif"))
Landuse_train= natsort.natsorted(glob(f"C:/Users/EBDL/Desktop/Land/re_train/train/landuse/*.tif"))
Pop_train= natsort.natsorted(glob(f"C:/Users/EBDL/Desktop/Land/re_train/train/popden/*.tif"))

In [15]:
def convert_npy(data_n,type):
    files=natsort.natsorted(glob(f"../data/geo_data/crop/{data_n}/*.tif"))
    tif_list=[]
    for i in range(len(files)):
        tmp = cv2.imread(files[i], cv2.IMREAD_COLOR)
        tmp = cv2.cvtColor(tmp, cv2.COLOR_BGR2RGB)
        tmp = cv2.resize(tmp, (24, 24))
        tmp = tmp / 255.0
        tif_list.append(tmp)

    data=np.array(tif_list)
    np.save(f'{data_n}_{type}.npy',data)